: 

In [ ]:
# If needed in a fresh environment:
# %pip install yfinance pandas

from __future__ import annotations

from datetime import datetime
from typing import Any, Dict

import pandas as pd
import yfinance as yf

# -----------------------------
# Configuration (February 2024)
# -----------------------------
TICKER = "AAPL"                  # change as needed
AS_OF_DATE = "2024-02-29"        # valuation snapshot date (end of Feb 2024)
FEB_START = "2024-02-01"
FEB_END_EXCLUSIVE = "2024-03-01" # yfinance end is exclusive


def _filter_cols_by_date(df: pd.DataFrame, as_of_date: str) -> pd.DataFrame:
    if df.empty:
        return df

    cutoff = datetime.strptime(as_of_date, "%Y-%m-%d").date()
    valid_cols = []

    for col in df.columns:
        # Typical yfinance columns are pandas Timestamps
        if hasattr(col, "date"):
            if col.date() <= cutoff:
                valid_cols.append(col)
            continue

        # Fallback if column is string date
        if isinstance(col, str):
            try:
                col_date = datetime.strptime(col, "%Y-%m-%d").date()
                if col_date <= cutoff:
                    valid_cols.append(col)
            except ValueError:
                pass

    return df[valid_cols] if valid_cols else pd.DataFrame()


def _get_mrq_value(df: pd.DataFrame, row_name: str) -> float:
    if df.empty or row_name not in df.index:
        return 0.0
    value = df.loc[row_name].iloc[0]
    return float(value) if pd.notna(value) else 0.0


def _get_ltm_value(df: pd.DataFrame, row_name: str, num_quarters: int = 4) -> float:
    if df.empty or row_name not in df.index:
        return 0.0
    subset = df.loc[row_name].iloc[:num_quarters]
    return float(subset.sum())


def fetch_yahoo_financials_for_valuation(ticker: str, as_of_date: str) -> Dict[str, Any]:
    stock = yf.Ticker(ticker)

    # Match the valuation-service connector's data sources
    q_inc = _filter_cols_by_date(stock.quarterly_financials, as_of_date)
    q_bal = _filter_cols_by_date(stock.quarterly_balance_sheet, as_of_date)
    ann_inc = _filter_cols_by_date(stock.financials, as_of_date)
    info = stock.info or {}

    # If quarterly BS is empty after filter, fallback to annual BS
    if q_bal.empty:
        q_bal = _filter_cols_by_date(stock.balance_sheet, as_of_date)

    # LTM flows with annual fallback
    has_4_quarters = (not q_inc.empty) and (len(q_inc.columns) >= 4)
    if has_4_quarters:
        revenues_base = _get_ltm_value(q_inc, "Total Revenue")
        ebit_reported_base = _get_ltm_value(q_inc, "Operating Income")
        rnd_expense = _get_ltm_value(q_inc, "Research And Development")
        tax_exp = _get_ltm_value(q_inc, "Tax Provision")
        pre_tax_inc = _get_ltm_value(q_inc, "Pretax Income")
        flow_basis = "LTM from quarterly statements"
    else:
        revenues_base = _get_mrq_value(ann_inc, "Total Revenue")
        ebit_reported_base = _get_mrq_value(ann_inc, "Operating Income")
        rnd_expense = _get_mrq_value(ann_inc, "Research And Development")
        tax_exp = _get_mrq_value(ann_inc, "Tax Provision")
        pre_tax_inc = _get_mrq_value(ann_inc, "Pretax Income")
        flow_basis = "annual fallback"

    # Revenue safeguard used in service
    if revenues_base <= 0:
        revenues_base = 1000.0

    # R&D history (newest -> oldest)
    if not ann_inc.empty and "Research And Development" in ann_inc.index:
        rnd_history = [
            float(x) if pd.notna(x) else 0.0
            for x in ann_inc.loc["Research And Development"].tolist()
        ]
    else:
        rnd_history = []

    # Balance sheet point-in-time (MRQ)
    if not q_bal.empty:
        stockholders_equity = _get_mrq_value(q_bal, "Stockholders Equity")
        total_equity_gross_mi = _get_mrq_value(q_bal, "Total Equity Gross Minority Interest")

        if total_equity_gross_mi > 0:
            book_equity = total_equity_gross_mi
            minority_interest = total_equity_gross_mi - stockholders_equity
        else:
            book_equity = stockholders_equity
            minority_interest = max(_get_mrq_value(q_bal, "Minority Interest"), 0.0)

        book_debt = _get_mrq_value(q_bal, "Total Debt")

        cash_equiv = _get_mrq_value(q_bal, "Cash Cash Equivalents And Short Term Investments")
        if cash_equiv == 0:
            cash_equiv = (
                _get_mrq_value(q_bal, "Cash And Cash Equivalents")
                + _get_mrq_value(q_bal, "Other Short Term Investments")
            )

        cross_holdings = _get_mrq_value(q_bal, "Investmentin Financial Assets")
        shares_outstanding = info.get("sharesOutstanding") or _get_mrq_value(q_bal, "Ordinary Shares Number")
    else:
        book_equity = 0.0
        minority_interest = 0.0
        book_debt = 0.0
        cash_equiv = 0.0
        cross_holdings = 0.0
        shares_outstanding = info.get("sharesOutstanding") or 0.0

    # Get February 2024 price history and month-end close as stock_price
    feb_prices = stock.history(start=FEB_START, end=FEB_END_EXCLUSIVE, interval="1d", auto_adjust=False)
    feb_prices = feb_prices.copy()
    if not feb_prices.empty:
        feb_prices.index = feb_prices.index.tz_localize(None) if feb_prices.index.tz is not None else feb_prices.index
        stock_price_as_of = float(feb_prices["Close"].iloc[-1])  # last trading day in Feb
    else:
        stock_price_as_of = float(info.get("currentPrice") or 0.0)

    # Tax rate handling
    country_to_marginal_tax = {
        "US": 0.21, "United States": 0.21, "IE": 0.125, "GB": 0.25,
        "CN": 0.25, "DE": 0.30, "JP": 0.3062
    }
    country = info.get("country", "US")
    marginal_tax_rate = country_to_marginal_tax.get(country, 0.25)

    if pre_tax_inc == 0:
        effective_tax_rate = 0.0
    else:
        raw_effective_rate = tax_exp / pre_tax_inc
        if raw_effective_rate < 0:
            effective_tax_rate = 0.0
        elif raw_effective_rate > 1:
            effective_tax_rate = marginal_tax_rate
        else:
            effective_tax_rate = float(raw_effective_rate)

    # Optional: risk-free rate from ^TNX
    tnx = yf.download("^TNX", period="1d", progress=False, auto_adjust=False)
    if not tnx.empty:
        risk_free_rate = float(tnx["Close"].iloc[-1]) / 100.0
    else:
        risk_free_rate = 0.04

    valuation_inputs = {
        "ticker": ticker,
        "as_of_date": as_of_date,
        "flow_basis": flow_basis,
        "revenues_base": float(revenues_base),
        "ebit_reported_base": float(ebit_reported_base),
        "rnd_expense": float(rnd_expense),
        "rnd_history": rnd_history,
        "book_equity": float(book_equity),
        "minority_interest": float(minority_interest),
        "book_debt": float(book_debt),
        "cash": float(cash_equiv),
        "cross_holdings": float(cross_holdings),
        "shares_outstanding": float(shares_outstanding) if shares_outstanding else 0.0,
        "stock_price": float(stock_price_as_of),
        "marginal_tax_rate": float(marginal_tax_rate),
        "effective_tax_rate": float(effective_tax_rate),
        "operating_leases_flag": "no",
        "risk_free_rate": float(risk_free_rate),
    }

    return {
        "valuation_inputs": valuation_inputs,
        "feb_2024_price_history": feb_prices,
        "quarterly_income_statement_filtered": q_inc,
        "quarterly_balance_sheet_filtered": q_bal,
        "annual_income_statement_filtered": ann_inc,
    }


result = fetch_yahoo_financials_for_valuation(TICKER, AS_OF_DATE)

print("Valuation inputs for", TICKER, "as of", AS_OF_DATE)
for k, v in result["valuation_inputs"].items():
    if k not in {"rnd_history"}:
        print(f"{k}: {v}")

print("\nFebruary 2024 daily prices (head):")
display(result["feb_2024_price_history"].head())

print("\nFebruary 2024 daily prices (tail):")
display(result["feb_2024_price_history"].tail())

: 